# Mass spectrometry data

The objective of this exercise is to read in raw peptide MSMS spectrum information and output a dataframe.
The .msp file can be downloaded [here](https://chemdata.nist.gov/download/peptide_library/libraries/cptaclib/2015/cptac2_mouse_hcd_selected.msp.tar.gz).

The information in this ASCII based text file is organized spectrum by spectrum.
The first line per spectrum provides formatted like this:

&emsp;<code>Name: sequence/charge_nmods_collisionenergy</code>

followed by a comment section which can be disregarded and the actual spectrum data which is tab-separated:

&emsp;<code>m/z&emsp;intensity&emsp;additional_info</code>

Spectra are separated by an empty line.

Code a function that returns two DataFrames or arrays containing the processed and filtered data. The first one should contain the spectrum information (n_spectra, n_m/z_features) and the second one the sequences per row (n_spectra).

Here are some general guidelines:

* The m/z values need to be binned to integer values (mathematically rounded), otherwise the dataframe size would get out of hand. This will allow for multiple values mapped to a single bin (e.g. if there are peaks at 145.1 and 145.2). Here, only the maximum of those peaks should be kept in the final dataframe.

* Rows that are all-zero should be dropped.

Your function should allow for selecting a range on the x-axis (m/z-range). All peaks outside this range can be disregarded. Furthermore, only spectra within a set collision energy range and a maximum sequence length should be contained in the output dataframe.

The faster your function runs, the better. I will time them all in the end.

In [4]:
import numpy as np
import pandas as pd
import timeit

In [18]:
def to_be_considered(line_, sequence, min_ce, max_ce, max_seq_len):
    conditions = True
    ce = float(line_.split("_")[-1][:-2])
    if ce < min_ce or ce > max_ce:
        conditions = False
    elif len(sequence) > max_seq_len:
        conditions = False
    return conditions

def get_description_parameters(line_, min_ce, max_ce, max_seq_len):
    name_ = str(line_[6:])
    sequence_ = line_.split("/")[0][6:]
    condition_ = to_be_considered(line_, sequence_, min_ce, max_ce, max_seq_len)
    return name_, sequence_, condition_


In [22]:
def msp_to_df(
    input_file,
    max_seq_len=30,
    min_ce=36,
    max_ce=40,
    mz_min=135,
    mz_max=1400,
):
    """
    Function to read spectrum data from .msp file and convert to dataframe.
    Args:
        input_file (str): path to .msp file
        max_seq_len (int): maximum acceptable sequence length
        min_ce (int): minimum collision energy of spectra to be included in df
        max_ce (int): maximum collision energy of spectra to be included in df
        mz_min (int): lower boundary for m/z to be included in df
        mz_max (int): lower boundary for m/z to be included in df

    Returns:
        df (pd.DataFrame or np.array): spectrum information within defined parameters [n_spectra, n_features]
        seqs (pd.DataFrame or np.array): sequences
    """
    col = ["Name", "Sequence"] + list(np.arange(mz_min,mz_max+1,1))
    ms_df = pd.DataFrame(columns = col)

    counter = 0
    
    for line in input_file:
        line = line.strip()
        if line[:4] == "Name":
            name, sequence, condition = get_description_parameters(line, min_ce, max_ce, max_seq_len)
            if condition == True:
                ms_df = ms_df.append(pd.Series(0, index=ms_df.columns), ignore_index=True)
                ms_df.at[counter, "Name"] = name
                ms_df.at[counter, "Sequence"] = sequence
                counter += 1

        if condition == True:
            if line[0:1] in ["0","1","2","3","4","5","6","7","8","9"]:
                m_z = round(float(line.split("\t")[0]),0)
                intensity = float(line.split("\t")[1])
                if mz_min <= m_z <= mz_max:
                    if ms_df.at[counter - 1, m_z] < intensity:
                        ms_df.at[counter - 1, m_z] = intensity
                
        # if counter == 10:
        #     break
    ms_df = ms_df.loc[~(ms_df.iloc[:, 2:]==0).all(axis=1)]
    return ms_df

In [23]:
with open ("../../data/cptac2_mouse_hcd_selected.msp", 'r') as in_file:
    ms_df = msp_to_df(in_file)
ms_df

,Name,Sequence,135,136,137,138,139,140,141,142,...,1391,1392,1393,1394,1395,1396,1397,1398,1399,1400
0,AAAFEDQENETVVVK/2_0_38.7eV,AAAFEDQENETVVVK,0,2851.7,0,0,0,0,1404.0,0,...,0,0,0,0,0,0,0,0,2797.6,0
1,AAGTIQTSVQEVNSK/2_0_37.3eV,AAGTIQTSVQEVNSK,0,2626.0,0,0,1553.4,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,AAGVSVEPFWPGLFAK/2_0_39.3eV,AAGVSVEPFWPGLFAK,0,59857.3,8295.8,0,3220.5,0,24351.6,2054.0,...,0,0,0,0,0,0,0,0,0,0
3,"AAHSQCAYSNPEGTVLLACEESR/3_2(5,C,CAM)(18,C,CAM)...",AAHSQCAYSNPEGTVLLACEESR,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"AAHSQCAYSNPEGTVLLACEESR/3_2(5,C,CAM)(18,C,CAM)...",AAHSQCAYSNPEGTVLLACEESR,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1946.0,0,0,0
5,AALLAELASLEADALR/2_0_38.1eV,AALLAELASLEADALR,0,30226.6,2175.2,0,2387.9,0,19422.5,1908.6,...,0,0,0,0,0,0,0,0,0,0
6,"AAPLSLCALTAVDQSVLLLKPEAK/3_1(6,C,CAM)_37eV",AAPLSLCALTAVDQSVLLLKPEAK,0,0,0,0,0,0,0,0,...,0,0,0,0,31817.1,0,0,0,0,25753.0
7,AAPQLPMEELVSLSK/2_0_37.8eV,AAPQLPMEELVSLSK,0,83365.3,21835.6,2636.3,0,2195.5,24785.9,0,...,0,0,0,0,0,0,10331.1,0,0,0
8,AASGFNATEDAQTLR/2_0_37.7eV,AASGFNATEDAQTLR,0,1285.7,0,0,0,0,1093.7,0,...,0,0,0,0,0,0,0,0,0,0
